## Cross-Validation and Bootstrap

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

We load the Auto data and split into train and validation

In [3]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

We can compute a linear model and check its MSE

In [4]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
Y_train = Auto_train.mpg
model = sm.OLS(Y_train, X_train)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,39.9055,1.009,39.537,0.0
horsepower,-0.1563,0.009,-17.333,0.0


In [5]:
X_valid = hp_mm.transform(Auto_valid)
Y_valid = Auto_valid.mpg
valid_pred = results.predict(X_valid)
np.mean((Y_valid - valid_pred)**2)

23.61661706966988

### Train-validation

In [6]:
#we difine a function that returns the MSE for a set of model, train and test samples
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    Y_train = train[response]
    
    X_test = mm.transform(test)
    Y_test = test[response]
    
    results = sm.OLS(Y_train, X_train).fit()
    test_pred = results.predict(X_test)
    
    return np.mean((Y_test - test_pred)**2)
    

In [7]:
#testing out new finction
MSE = np.zeros(3)

for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower',degree)], 'mpg', Auto_train, Auto_valid)

MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
#if we set a new validation sample the results should change a bit
Auto_train , Auto_valid = train_test_split(Auto, test_size=196, random_state=3)

MSE = np.zeros(3)

for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower',degree)], 'mpg', Auto_train, Auto_valid)

MSE


array([20.75540796, 16.94510676, 16.97437833])

Both results suggest that a quadratic model is more accurate than a linear one

### Cross-Validation

Cross validations requires conneting output from different libraries, the wrapper sklearn_sm from ISLP allows us to use the cross-validation tools of sklearn with the models fit by statsmodels

In [9]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0]) # arguments are an apropiate object(that can be .fit() .predict()), an array of features X, a response Y, cv provides the number of folds
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929212

We can again loop this for more complex polynomial models to se how they perform

In [10]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1,6)):
    X = np.power.outer(H,np.arange(d+1)) #outer() explained in next code box, it adds polynomial factors
    M_CV = cross_validate(M,X,Y,cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error    

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

In [11]:
A = np.array([3,5,9])
B = np.array([2,4])
np.add.outer(A, B), np.power.outer(A,B) # applies the second argument to the first based on the fisrt method(add, min,power)

(array([[ 5,  7],
        [ 7,  9],
        [11, 13]]),
 array([[   9,   81],
        [  25,  625],
        [  81, 6561]]))

We can also set a smaller K-fold 

In [12]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0) #setting the random state ensures the same splits for each degree

for i, d in enumerate(range(1,6)):
    X = np.power.outer(H,np.arange(d+1))
    M_CV = cross_validate(M,X,Y,cv=cv) # we set cv=cv here
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error  

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

We observe similar results , smaller number of kfolds can make the run time much faster in certain ocations

The cross_validate() function is very flexible and can take different spliting mechanisims not only Kfold

In [13]:
validation = ShuffleSplit(n_splits=1,test_size=196,random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'],axis=1),Auto['mpg'], cv=validation);
results['test_score']

array([23.61661707])

In [14]:
validation = ShuffleSplit(n_splits=10,test_size=196,random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'],axis=1),Auto['mpg'], cv=validation);
results['test_score'].mean(), results['test_score'].std()

(23.80223266103416, 1.42184509410918)

### Bootstrap

In [15]:
Portfolio = load_data('Portfolio')
def alpha_func(D,idx): # we define a function for estimating the alpha based on the data
    cov_ = np.cov(D[['X','Y']].loc[idx],rowvar=False) #rowvar specifies that the data is in the columns
    return ((cov_[1,1] - cov_[0,1])/(cov_[0,0]+cov_[1,1]-2*cov_[0,1]))
    

In [16]:
alpha_func(Portfolio, range(100)) #computes alpha for the 100 obs in the data

0.57583207459283

In [17]:
rng = np.random.default_rng(0)

In [18]:
alpha_func(Portfolio,rng.choice(100,100,replace=True)) #computes alpha for 100 random obs in the dataset, picked with replacement

0.6074452469619004

In [19]:
def boot_SE(func, D, n=None, B=1000, seed=0): # we can put it all together in one formula to compute the standard error of the estimated alpha
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,n,replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)
    


In [20]:
boot_SE(alpha_func,Portfolio)

0.09118176521277699

In [21]:
Auto = load_data('Auto')

In [22]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_,X_).fit().params

In [23]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg') # this line creates a version of the bool_OLS function where the model matrix and response are fixed to the specified values

In [24]:
rng = np.random.default_rng(0)
n = Auto.shape[0]
np.array([hp_func(Auto, rng.choice(n,n,replace=True)) for _ in range(10)]) # the new function can be used to create bootstrap examples of beta0 and beta1

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [25]:
# we can still use boot_SE to compute the standard errors
hp_SE = boot_SE(hp_func,Auto,B=1000, seed=10)
hp_SE

intercept     0.848807
horsepower    0.007352
dtype: float64

In [26]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [27]:
quad_model = MS([poly('horsepower',2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func,Auto)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [28]:
M = sm.OLS(Auto['mpg'],quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64